# **프로젝트 개요**
## **1. 배경**
안녕하세요 여러분! 😀 신용카드 사기 거래 탐지 AI 경진대회에 오신 것을 환영합니다.

신용카드 회사는 사기성 신용카드 거래를 탐지하여 고객이 구매하지 않은 항목에 대해 요금을 청구하지 않는 것이 중요하며, 이러한 문제를 해결하기 위한 사기 거래를 사전에 탐지할 수 있는 도구가 필요합니다.

그러나 매일 누적되는 방대한 양의 신용카드 거래 데이터 속에서 사기 거래는 극히 일부이며, 방대한 양의 사기 거래 케이스의 데이터를 구축하기에는 너무나도 많은 시간이 소요될 것입니다. 또한 모든 데이터에 대해서 사기 거래 여부를 확인하고 기록하는 작업(Labeling)은 많은 비용을 필요로 합니다.

만약, **신용카드 사기 거래의 여부를 모르는 방대한 데이터로도 사기 거래를 탐지할 수 있는 AI 모델(Unsupervised Anomaly Detection)을 개발할 수 있다면** 데이터 구축에 필요한 시간, 비용의 문제도 모두 해결할 수 있을 것입니다.

추가로, **기존에 구축된 신용카드 사기 거래 여부를 알 수 있는 데이터셋(Validation)은 데이커분들이 이러한 AI 모델을 개발하고 자체적으로 성능을 평가**해보는 것에 있어서 큰 도움을 줄 것입니다.

따라서 데이터 구축관련 비용 문제를 해결하고, 신용카드 사기 거래 역시 완벽히 탐지해낼 수 있는 AI 모델을 개발해주세요!

## **2. 주제**
비식별화된 신용카드 거래 데이터로부터 사기 거래를 탐지하는 AI 솔루션 개발

## **3. 주최 / 주관**
주최 / 주관: 데이콘

## **초기 세팅**
* Google Colab 환경

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"
!unzip "open.zip"

## **결과**
* Validation F1 Score : 70%
* 점수 : 68

## **Code**
### **Import**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

### **Data Load**

In [ ]:
train_df = pd.read_csv('./train.csv') # Train
train_df.head()

In [ ]:
val_df = pd.read_csv('./val.csv') # Validation
val_df.head()

### **Train / Validation Feature 분포 확인**

In [ ]:
train_df.drop(columns=['ID']).hist(bins = 50, figsize =(20,20))
plt.show()

In [ ]:
val_df.drop(columns=['ID','Class']).hist(bins = 50, figsize =(20,20))
plt.show()

### **Validation set 사기 거래 비율**
* **Validation set의 사기 거래 비율이 다른 데이터 집합에서도 비슷하게 발생할 것이라고 가정**

In [ ]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

### **Model Define & Fit**

In [ ]:
# Train dataset은 Label이 존재하지 않음
train_x = train_df.drop(columns=['ID']) # InputData(ID 제외)    

In [ ]:
# 가설 설정 
# Train Dataset도 Validation Dataset과 동일한 비율로 사기거래가 발생했을 것이다. 
# => model parameter값을 val_contamination으로
model = IsolationForest(n_estimators = 125, max_samples = len(train_x), contamination = val_contamination, random_state=42, verbose=0)
model.fit(train_x)

### **Evaluation : Validation set**

In [ ]:
def get_pred_label(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1,1, model_pred)

    return model_pred

In [ ]:
val_x = val_df.drop(columns=['ID','Class']) #Input Data
val_y = val_df['Class'] # Label

val_pred = model.predict(val_x)
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y,val_pred,average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y,val_pred))

### **Inference : Test set**

In [ ]:
test_df = pd.read_csv('./test.csv')
test_df.head()

In [ ]:
test_x = test_df.drop(columns=['ID'])

In [ ]:
test_scaled = Scaler.fit_transform(test_x)
test_x = pd.DataFrame(test_scaled, columns = test_x.columns, index=list(test_x.index.values))

In [ ]:
test_pred = model.predict(test_x) # model prediction
test_pred = get_pred_label(test_pred)

### **Submission**

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

In [ ]:
submit['Class'] = test_pred
submit.to_csv('./submit.csv',index=False)

In [ ]:
submit.head()